In [ ]:
%matplotlib inline

In [ ]:
import os
import matplotlib.pyplot as plt
import polars as pl
from pathlib import Path

import coola
from coola.utils import str_mapping

In [ ]:
plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (16, 5)

In [ ]:
from arctix.dataset import epic_kitchen_100
from arctix.dataset.epic_kitchen_100 import Column

In [ ]:
# Define the path where to store/load the raw data
data_path = Path(os.environ["ARCTIX_DATA_PATH"]).joinpath("epic_kitchen_100")

In [ ]:
# config = pl.Config.set_tbl_cols(-1).set_tbl_rows(10).set_tbl_width_chars(100).set_fmt_table_cell_list_len(100)

## Load raw data

In [ ]:
data_raw, metadata = epic_kitchen_100.fetch_data(data_path, split="train")
data_raw

In [ ]:
data_raw.describe()

## Analyze raw data

In [ ]:
group_len = data_raw.group_by([Column.VIDEO_ID]).len()
group_len

In [ ]:
max_len = group_len.select(pl.col("len")).max().item()
min_len = group_len.select(pl.col("len")).min().item()
nbins = max_len - min_len + 1
print(f"min={min_len}  max={max_len}  nbins={nbins}")

fig, ax = plt.subplots()
_ = ax.hist(group_len.select(pl.col("len")), bins=nbins, range=[min_len, max_len])

_ = ax.set_title("distribution of sequence length")
_ = ax.set_xlabel("sequence length")
_ = ax.set_ylabel("count")

In [ ]:
col = Column.VERB
num_actions = data_raw.select(pl.col(col)).n_unique()
fig, ax = plt.subplots()
_ = ax.hist(data_raw.select(pl.col(col)), bins=num_actions)

_ = ax.set_title("distribution of actions")
_ = ax.set_xlabel("action")
_ = ax.set_ylabel("count")
_ = ax.tick_params(axis="x", labelrotation=90)

## Prepare data

In [ ]:
data, metadata = epic_kitchen_100.prepare_data(data_raw, split="validation")
data

In [ ]:
print(str_mapping(metadata))

In [ ]:
arrays = to_array(data)

In [ ]:
print(coola.summary(arrays))

In [ ]:
print(str_mapping(arrays))

In [ ]:
data = epic_kitchen_100.load_event_data(data_path.joinpath("EPIC_100_train.csv"))
data

In [ ]:
data.describe()